In [101]:
# 알라딘 책 정보 크롤링으로 한번 더 연습
import time
import bs4
import csv
import datetime
import urllib
import json
import pymysql

In [102]:
aladin_url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=TodayHot&BranchType=1&CID=1'
bsObj = bs4.BeautifulSoup(urllib.request.urlopen(aladin_url).read(), 'html.parser')
book_container = bsObj.find('form', {'name':'Myform'})
book_container

<form action="/shop/wbasket.aspx" id="Myform" method="post" name="Myform">
<!--도서관사서 노출 구간-->
<div class="megaseller_clbox megaseller_sp2"> </div>
<div class="list_add_btn_box"><input checked="checked" name="chkAllCheck" style="display:none;" type="checkbox"/>
<div><button class="list_add_btn" onclick="revcheck_Form(this)" title="리스트의 모든상품을 선택합니다." type="button">전체선택</button></div>
<div><button class="list_add_btn" onclick="return SafeBasket_ListOneAddByAjax(null, this, {top: 25, left:-50})" title="체크한 도서를 모두 장바구니에 담습니다." type="button">장바구니 담기</button></div><div><button class="list_add_btn" id="wa_BestV3_Wa_btn_basket_add_all1" onclick="return AddSafeBasketAll();" title="체크한 상품를 모두 보관함에 담습니다." type="button">보관함 담기</button></div><div><button class="list_add_btn" onclick="return AddMyListAll();" title="체크한 상품를 모두 마이리스트에 담습니다." type="button">마이리스트 담기</button></div></div>
<div class="ss_book_box" itemid="358328336">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td a

In [103]:
#데이터 전처리
book_name = book_container.find_all('a',class_='bo3')
book_discounted_price = book_container.find_all('span',class_='ss_p2')
# book_name = book_container.find_all('a',{'class'='bo3'}) 와 같음, 커스텀 attribute 가 아니면 지원함

book_data = []
for book,price in zip(book_name,book_discounted_price):
    title = book.text # 제목
    authors_list = book.parent.next_sibling.next_sibling.find_all('a', href=lambda x: x and x.startswith('/Search')) # 지은이
    authors = ','.join([author.text for author in authors_list])
    original_price = f"{price.previous_sibling.previous_sibling.text}원" # 원가
    discounted_price = price.text # 할인가
    print(f"책 제목:{title}\n지은이:{authors}\n원가:{original_price}\n할인가:{discounted_price}")
    book_data.append({"title":title, "author":authors, "oPrice":original_price, "dPrice":discounted_price})

책 제목:그 개와 혁명
지은이:예소연,김기태,문지혁,서장원,정기현,최민우
원가:17,500원
할인가:15,750원
책 제목:오랜 거짓말이 끝나는 날에
지은이:이누준,김진환
원가:17,800원
할인가:16,020원
책 제목:사랑은 하트 모양이 아니야
지은이:김효인
원가:12,000원
할인가:10,800원
책 제목:이상한 집 2
지은이:우케쓰,김은모
원가:18,000원
할인가:16,200원
책 제목:이별은 모차르트
지은이:나카야마 시치리,문지원
원가:17,800원
할인가:16,020원
책 제목:초판본 싯다르타
지은이:헤르만 헤세,박진권
원가:2,980원
할인가:2,690원
책 제목:J.R.R. 톨킨 동화 선집 세트 - 전5권
지은이:존 로널드 루엘 톨킨,크리스티나 스컬,웨인 G. 해먼드,벌린 플리거,폴린 베인즈,존 로널드 루엘 톨킨,김보원,이미애
원가:128,000원
할인가:115,200원
책 제목:겨울 여행
지은이:자우메 카브레,권가람
원가:15,000원
할인가:13,500원
책 제목:초판본 노인과 바다
지은이:어니스트 헤밍웨이,이수정
원가:2,980원
할인가:2,690원
책 제목:소년
지은이:가와바타 야스나리,정수윤
원가:16,800원
할인가:15,120원
책 제목:최애의 살인
지은이:엔도 가타루,전선영
원가:16,800원
할인가:15,120원
책 제목:우리 집에 왜 왔어?
지은이:정해연
원가:15,000원
할인가:13,500원
책 제목:입에 대한 앙케트
지은이:세스지,오삭
원가:6,500원
할인가:5,850원
책 제목:초판본 이방인
지은이:알베르 카뮈,최헵시바
원가:2,980원
할인가:2,690원
책 제목:70세 사망법안, 가결
지은이:가키야 미우,김난주
원가:16,800원
할인가:15,120원
책 제목:초판본 변신
지은이:프란츠 카프카,한영란
원가:2,980원
할인가:2,690원
책 제목:데이터 상속인
지은이:한상운
원가:17,000원
할인가:15,300원
책 제목:심장보다 단단한 토마토 한 알
지은이:고선경
원가:12,000원
할인가:10,800원
책 제

In [104]:
#csv 저장
with open(r"./book_data.csv","w",encoding="utf-8",newline="") as csvfile:
    # Dictionary 를 쓸거니까 DictWriter
    bookwriter = csv.DictWriter(csvfile, fieldnames=["title","author","oPrice","dPrice"])
    bookwriter.writerows(book_data)
#json 저장
with open(r"./book_data.json","w",encoding="utf-8",newline="") as jsonfile:
    json.dump(book_data, jsonfile, ensure_ascii=False, indent=4)

In [106]:


db = pymysql.connect(host="localhost", user="webuser", passwd="webuser", db="webdb")

cursor = db.cursor()

listed_book_data = [(book['title'],book['author'],book['oPrice'],book['dPrice']) for book in book_data]

query = "INSERT INTO booksAladin (title, author, oPrice, dPrice) VALUES (%s, %s, %s, %s)"

try:
    # execute , execute many 둘 다 tuple, list 의 형태여야 한다. 그래야 순서에 맞춰 values 에 넣을 수 있음
    # 그래서 위에서 dictionary 인 book 을 tuple 형태로 바꿔 리스트로 다시만듦
    cursor.executemany(query,listed_book_data)
    db.commit()
    print("db 저장 완료")
except Exception as e:
    db.rollback()
    print(e)
cursor.close()
db.close()

db 저장 완료
